<h1>Imports</h1>

In [1]:
from pathlib import Path

import numpy as np
import sklearn as slearn
import pandas as pd
import matplotlib.pyplot as plt
import math
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import fbeta_score
from sklearn.metrics import classification_report
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from imblearn.combine import SMOTETomek

import glob
import os

Using TensorFlow backend.


In [2]:
pd.set_option("display.max_columns", 100)
%matplotlib inline

In [3]:
RANDOM_SEED = 6

<h1>Dataset Importing</h1>

In [4]:
DATA_PATH = Path.cwd() / "../../datasets"

In [5]:
training_df = pd.read_csv(DATA_PATH / "train.csv", index_col="tripid")
training_df.head()

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,label
tripid,,,,,,,,,,,,,
189123628,10.5,834.0,56.0,0.0000,64.0,11/1/2019 0:20,11/1/2019 0:34,6.86252,79.8993,6.90330,79.8783,270.32,correct
189125358,10.5,791.0,47.0,0.0000,134.0,11/1/2019 0:56,11/1/2019 1:09,6.88589,79.8984,6.91373,79.8923,197.85,correct
189125719,10.5,1087.0,80.0,0.0000,61.0,11/1/2019 1:08,11/1/2019 1:26,6.90839,79.8651,6.93669,79.9146,301.64,correct
189127273,10.5,598.0,271.0,15.6638,68.0,11/1/2019 2:27,11/1/2019 2:37,6.92570,79.8895,6.92748,79.8971,82.30,correct
189128020,NaN,NaN,NaN,NaN,NaN,11/1/2019 3:34,11/1/2019 3:51,6.87441,79.8615,6.84478,79.9290,358.39,correct


In [6]:
training_df = training_df.dropna(subset=['fare'])

<h1>Data Preprocessing</h1>

<h2>Data Conversions</h2>

In [7]:
# Converting date time objects from Objects to datetime64
training_df['pickup_time'] = pd.to_datetime(training_df['pickup_time'], format="%m/%d/%Y %H:%M")
training_df['drop_time'] = pd.to_datetime(training_df['drop_time'], format="%m/%d/%Y %H:%M")

<h2>Adding Features</h2>
Adding features deemed useful from Feature Engineering

In [8]:
durations = []
for index,row in training_df.iterrows():
  provided_duration = row['duration'] #first row of location.lat column here
  if math.isnan(provided_duration) or provided_duration <= 0 :
    time_dif = (row['drop_time'] - row['pickup_time']).seconds
    if(time_dif == 0):
        time_dif = 60
    durations.append(time_dif)
  else :  
    durations.append(provided_duration)

training_df.insert(4,"time_dif",durations)



In [9]:
def dist_from_coordinates(lat1, lon1, lat2, lon2):
  R = 6371  # Earth radius in km

  #conversion to radians
  d_lat = np.radians(lat2-lat1)
  d_lon = np.radians(lon2-lon1)

  r_lat1 = np.radians(lat1)
  r_lat2 = np.radians(lat2)

  #haversine formula
  a = np.sin(d_lat/2.) **2 + np.cos(r_lat1) * np.cos(r_lat2) * np.sin(d_lon/2.)**2

  haversine = 2 * R * np.arcsin(np.sqrt(a))

  return haversine

In [10]:
training_df = training_df.assign(timeOfDay=pd.cut(training_df.pickup_time.dt.hour,[-1, 8, 20, 24],labels=['dawn','day', 'night']))

In [11]:
new_column = []                    #empty column for distance
for index,row in training_df.iterrows():
  lat1 = row['pick_lat'] #first row of location.lat column here
  lon1 = row['pick_lon'] #first row of location.long column here
  lat2 = row['drop_lat'] #second row of location.lat column here
  lon2 = row['drop_lon'] #second row of location.long column here
  value = dist_from_coordinates(lat1, lon1, lat2, lon2)  #get the distance
  new_column.append(value)   #append the empty list with distance values

training_df.insert(4,"distance",new_column)

In [12]:
training_df['avg_speed'] = (training_df['distance'] /  ( training_df['time_dif']) * 3600 )

In [13]:
training_df = training_df.drop(index=190167541)

In [14]:
training_df.nlargest(100,'avg_speed')

,additional_fare,duration,meter_waiting,meter_waiting_fare,distance,time_dif,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,label,timeOfDay,avg_speed
tripid,,,,,,,,,,,,,,,,,
206894975,10.5,33.0,32.0,0.0000,33.601116,33.0,102.0,2020-01-04 02:38:00,2020-01-04 02:38:00,6.90634,79.9005,6.60862,79.9526,171.49,incorrect,dawn,3665.576301
194344562,10.5,27.0,0.0,0.0000,25.653969,27.0,144.0,2019-11-23 17:20:00,2019-11-23 17:20:00,6.93399,79.9821,7.14629,80.0731,134.41,incorrect,day,3420.529206
204570020,10.5,33.0,21.0,0.0000,25.903941,33.0,48.0,2019-12-27 21:43:00,2019-12-27 21:43:00,6.84846,79.9141,6.78734,80.1405,1264.83,correct,night,2825.884515
204281688,21.0,35.0,0.0,0.0000,25.210253,35.0,464.0,2019-12-26 17:32:00,2019-12-26 17:32:00,7.08468,79.8876,6.85861,79.8703,1100.31,correct,day,2593.054569
202225769,10.5,32.0,6.0,0.0000,21.952446,32.0,93.0,2019-12-19 23:05:00,2019-12-19 23:06:00,6.91402,79.8786,6.90885,80.0774,104.60,incorrect,night,2469.650222
189749297,10.5,26.0,24.0,0.0000,17.237800,26.0,26.0,2019-11-03 21:56:00,2019-11-03 21:56:00,6.90625,79.9275,6.75374,79.8995,839.68,incorrect,night,2386.772335
196943952,10.5,27.0,24.0,0.0000,17.048079,27.0,40.0,2019-12-01 10:20:00,2019-12-01 10:21:00,6.93344,79.8548,6.87023,79.9955,90.28,incorrect,day,2273.077221
211538511,10.5,24.0,21.0,0.0000,14.674406,24.0,28.0,2020-01-22 21:23:00,2020-01-22 21:24:00,6.85305,79.9190,6.92109,80.0329,80.25,incorrect,night,2201.160829
193258422,10.5,25.0,10.0,0.0000,14.763471,25.0,116.0,2019-11-19 21:02:00,2019-11-19 21:02:00,6.92725,79.8451,7.03410,79.9245,82.24,incorrect,night,2125.939838


<h1>Model Training</h1>

In [15]:
# Set of new columns
new_columns = ['additional_fare', 'time_dif', 'meter_waiting', 'meter_waiting_fare',
       'meter_waiting_till_pickup']

In [16]:
# Extract the useful features deemed in feature engineering
features_classifier = training_df[new_columns]
label_classifier = training_df[['label']]
mapping = {'correct': 1, 'incorrect': 0}
label_classifier = label_classifier.replace({'label':mapping})

In [17]:
features_classifier.dtypes

additional_fare              float64
time_dif                     float64
meter_waiting                float64
meter_waiting_fare           float64
meter_waiting_till_pickup    float64
dtype: object

In [18]:
numerical_features = features_classifier.columns[features_classifier.dtypes == "float64"].values
categorical_features = features_classifier.columns[features_classifier.dtypes == "category" ].values

In [19]:
kf = StratifiedKFold(n_splits=100)

In [20]:
numeric_preprocessing_steps = Pipeline([
    ('simple_imputer', SimpleImputer(strategy='median'))
])

non_numeric_preprocessing_steps = Pipeline([
    ('simple_imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot_encoder', OneHotEncoder(handle_unknown='ignore')),
#     ('label_encoder', LabelEncoder())
])

## create preprocessor stage of the final pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('non_numeric', non_numeric_preprocessing_steps, categorical_features),
        ('numeric', numeric_preprocessing_steps, numerical_features)
    ],
    remainder = 'drop'
)

In [21]:
X_train, X_eval, y_train, y_eval = train_test_split(features_classifier, label_classifier, test_size=0.3, shuffle=True,stratify=label_classifier)


In [22]:
param_grid = [
        {
            'activation' : ['identity', 'logistic', 'tanh', 'relu'],
            'solver' : ['lbfgs', 'sgd', 'adam'],
            'hidden_layer_sizes': [
             (1,),(2,),(3,),(4,),(5,),(6,),(7,),(8,),(9,),(10,),(11,), (12,),(13,),(14,),(15,),(16,),(17,),(18,),(19,),(20,),(21,)
             ]
        }
       ]

In [23]:
classifier = RandomForestClassifier()

In [ ]:
for train_index, test_index in kf.split(X_train,y_train):
    X_train_fold, X_eval_fold = X_train.iloc[train_index], X_train.iloc[test_index]
    y_train_fold, y_eval_fold = y_train.iloc[train_index], y_train.iloc[test_index]
    # Transform training data columns 
    preprocessed_train_features = preprocessor.fit_transform(X_train_fold)
    preprocessed_train_features_data_frame = pd.DataFrame(data=preprocessed_train_features, columns=new_columns)
    
    
    ## Train the model
    classifier.fit(preprocessed_train_features_data_frame, y_train_fold.values.ravel())

In [ ]:
preprocessed_test_features = preprocessor.fit_transform(X_eval)
preprocessed_test_features_data_frame = pd.DataFrame(data=preprocessed_test_features, columns=new_columns)

In [ ]:
# Predict for the evaluation set
print("Training Accuracy: %.2f" % (classifier.score(preprocessed_test_features_data_frame, y_eval)*100), "%")
y_pred = classifier.predict(preprocessed_test_features_data_frame)

In [ ]:
(pd.Series(classifier.feature_importances_, index=new_columns)
   .nlargest(30)
   .plot(kind='barh'))

In [ ]:
confusion_matrix = confusion_matrix(y_eval, y_pred)
confusion_matrix

In [ ]:
print(classification_report(y_eval, y_pred))

<h1>Model Validation</h1>

<h2>Loading the test data set</h2>

In [ ]:
test_set = pd.read_csv(DATA_PATH / "test.csv", index_col="tripid")
test_set.head()

In [ ]:
test_set.isna().sum()

<h2>Feature Addition for the Test Dataset</h2>

In [ ]:
test_set['pickup_time'] = pd.to_datetime(test_set['pickup_time'], format="%m/%d/%Y %H:%M")
test_set['drop_time'] = pd.to_datetime(test_set['drop_time'], format="%m/%d/%Y %H:%M")

In [ ]:
test_set = test_set.assign(timeOfDay=pd.cut(test_set.pickup_time.dt.hour,[-1, 8, 20, 24],labels=['dawn','day', 'night']))

In [ ]:
new_column = []                    #empty column for distance
for index,row in test_set.iterrows():
  lat1 = row['pick_lat'] #first row of location.lat column here
  lon1 = row['pick_lon'] #first row of location.long column here
  lat2 = row['drop_lat'] #second row of location.lat column here
  lon2 = row['drop_lon'] #second row of location.long column here
  value = dist_from_coordinates(lat1, lon1, lat2, lon2)  #get the distance
  new_column.append(value)   #append the empty list with distance values

test_set .insert(4,"distance",new_column)

In [ ]:
durations = []
for index,row in test_set.iterrows():
  provided_duration = row['duration'] #first row of location.lat column here
  if math.isnan(provided_duration) or provided_duration <= 0 :
    time_dif = (row['drop_time'] - row['pickup_time']).seconds
    if(time_dif == 0):
        time_dif = 60
    durations.append(time_dif)
  else :  
    durations.append(provided_duration)

test_set.insert(4,"time_dif",durations)



In [ ]:
test_set['avg_speed'] = (test_set['distance'] /  ( test_set['time_dif']) * 3600 )

In [ ]:
test_set

In [ ]:
test_features = test_set[['additional_fare', 'time_dif', 'meter_waiting', 'meter_waiting_fare',
       'meter_waiting_till_pickup','distance','avg_speed','fare']]
preprocessed_test_features = preprocessor.fit_transform(test_features)
preprocessed_test_features_data_frame = pd.DataFrame(data=preprocessed_test_features, columns=new_columns)

<h2>Fare prediction and correctness prediction using Test Dataset </h2>

In [ ]:
predicted_labels = classifier.predict(preprocessed_test_features_data_frame)

<h2>Writing to the Submission File</h2>

In [ ]:
submission_set = pd.read_csv(DATA_PATH / "sample_submission.csv", index_col="tripid")
submission_set.head()

submission_set['prediction']= predicted_labels

In [ ]:
%%javascript
var kernel = IPython.notebook.kernel;
var thename = window.document.getElementById("notebook_name").innerHTML;
var command = "theNotebook = " + "'"+thename+"'";
kernel.execute(command);

In [ ]:
filename = '../../submissions/'+theNotebook+'/teamCluster_submission_{%i}.csv'
dirname = '../../submissions/'+theNotebook
fileversion = 1

if not os.path.exists(dirname):
    os.makedirs(dirname)
while glob.glob(filename.replace('{%i}',str(fileversion))) :
    fileversion+=1
submission_set.to_csv(filename.replace('{%i}',str(fileversion)), index=True)
print("Completed!")

In [ ]:
submission_set['prediction'].idxmin()

In [ ]:
submission_set['prediction'].value_counts()